In [1]:
import sys
sys.path.append("./")
sys.path.append("./hw2/stud/")
from torch.utils.data import DataLoader
import config
# from tqdm.auto import tqdm
import numpy as np
from utils import *
from load import *
from model import ConSecModel
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
# from trainer import Trainer


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/home/francesco/Desktop/University/MNLP/nlp2023-hw2/hw2/stud


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# FINE_TRAIN_DATA
# FINE_VAL_DATA
# FINE_TEST_DATA



In [3]:
print(config.DEVICE)

cpu


In [4]:
seed_everything(config.SEED)

In [5]:

train_dataset = FineGrainedDataset(config.FINE_TRAIN_DATA)
val_dataset = FineGrainedDataset(config.FINE_VAL_DATA)
test_dataset = FineGrainedDataset(config.FINE_TEST_DATA)

train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

mapping = load_map(config.MAP_PATH)


/home/francesco/Desktop/University/MNLP/nlp2023-hw2/hw2/stud


In [6]:
# sense embeddigs
sense_embeddings = compute_definition_embeddigns()

# save sense embeddings in a file
np.save(config.SENSE_EMBEDDINGS_PATH, sense_embeddings)
# # model
# model = ConSecModel(num_senses=sense_embeddings)

# # Optimizer
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# # Loss
# criterion = torch.nn.CrossEntropyLoss()



FileNotFoundError: [Errno 2] No such file or directory: '../../data/intermediate/sense_embeddings.npy'

In [8]:
trainer = Trainer(max_epochs=10, callbacks=[ModelCheckpoint(monitor='val_loss')])
# trainer.train_epoch()
trainer.fit(model, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name             | Type             | Params
------------------------------------------------------
0 | bert             | RobertaModel     | 124 M 
1 | sense_embeddings | Embedding        | 3.4 M 
2 | fc               | Linear           | 3.4 M 
3 | loss_fn          | CrossEntropyLoss | 0     
------------------------------------------------------
131 M     Trainable params
0         Non-trainable params
131 M     Total params
526.101   Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/francesco/miniconda3/envs/NLP/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/francesco/miniconda3/envs/NLP/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_23220/4116552099.py", line 3, in <module>
    trainer.fit(model, train_loader, val_loader)
  File "/home/francesco/miniconda3/envs/NLP/lib/python3.11/site-packages/pytorch_lightning/trainer/trainer.py", line 531, in fit
    call._call_and_handle_interrupt(
  File "/home/francesco/miniconda3/envs/NLP/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py", line 42, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/francesco/miniconda3/envs/NLP/lib/python3.11/site-packages/pytorch_lightning/trainer/trainer.py", line 570, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/home/francesco/miniconda3/envs/NLP/lib/python3.11/site-packages/pytorch_lightning/traine

In [ ]:
# sentence = "I went to the bank to deposit my money"
# words = sentence.split()
# candidates = {4: ["river", "river2", "money"], 8: ["money", "richness"]}
# definitions = {"river": "a large natural stream of water (larger than a creek)", "river2" : "a financial institution that accepts deposits and channels the money into lending activities", "money": "the most common medium of exchange", "richness": "the property of being extremely abundant"}
# senses = {4: 2, 8: 0}

# new_input_sequences = []
# for idx in candidates.keys():
#     input_sequence = ['<s>'] + words[:idx] + ['<d>', words[idx], '</d>'] + words[idx+1:]
#     for candidate in candidates[idx]:
#         input_sequence.extend(['<def>'] + definitions[candidate].split())
#     for other_idx in candidates.keys():
#         if other_idx != idx:
#             input_sequence.extend(['<GT>'] + definitions[candidates[other_idx][senses[other_idx]]].split())
#     input_sequence.append('</s>')
#     new_input_sequences.append(input_sequence)